# Data 422 Project

Data wrangling, s2 2019

#### Joo-Hyun Ahn ( ID: 84701204)
#### Jia Zhiying ( ID: 54801975)
#### Changmin Moon ( ID: 72452116)
#### Komlev Viacheslac ( ID: 98419435)

## Contents

##### Step_1: Get the budget and revenue of the movies by web scrap using R
##### Step_2: Get the variable 'movieID' for merging with IMDB dataset by API using R
##### Step_3
##### Step_4
##### Step_5

### Step_1. Data gathering
#### 1. Web crawling
We have 8 large datasets from IMDB, but it does not have the budget and revenue of the movies. Hence, we need to do web scraping from the-numbers.com which have financial data.

In [1]:
# Call relevant libraries
library(tidyverse)
library(rvest)
library(magrittr)
library(purrr)
library(glue)
library(lubridate)
library(readr)
library(visdat)
library(skimr)
library(glue)

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
√ ggplot2 3.2.1     √ purrr   0.3.2
√ tibble  2.1.3     √ dplyr   0.8.3
√ tidyr   0.8.3     √ stringr 1.4.0
√ readr   1.3.1     √ forcats 0.4.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Loading required package: xml2

Attaching package: 'rvest'

The following object is masked from 'package:purrr':

    pluck

The following object is masked from 'package:readr':

    guess_encoding


Attaching package: 'magrittr'

The following object is masked from 'package:purrr':

    set_names

The following object is masked from 'package:tidyr':

    extract


Attaching package: 'glue'

The following object is masked from 'package:dplyr':

    collapse


Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date


Attaching package: 'skimr'

The following object is maske

In [2]:
# Designate the values which are at the end of the targeting URLs
movie_budgets_rankings <- list(url = seq(1, 5801, by=100))

# Glue the values to the common URL which is before the last values
movie_df_url_links <- c(url = glue("https://www.the-numbers.com/movie/budgets/all/{movie_budgets_rankings$url}"))

# Take a look the glued URLs
movie_df_url_links %>% glimpse()

 Named chr [1:59] "https://www.the-numbers.com/movie/budgets/all/1" ...
 - attr(*, "names")= chr [1:59] "url1" "url2" "url3" "url4" ...


In [3]:
# Make a function which scrapes a table from an inputted URL and returns a data frame 
get_movie_df_info <- function(movie_url){
    read_page <- read_html(movie_url) %>% html_table(fill = TRUE) # Scrape a table from a URL
    read_page <- as.data.frame(read_page) # Set as a data frame
    read_page[,1] <- as.numeric(gsub("\\D", "", read_page[,1])) # Change values in 'Rank' column into numeric values 
    read_page[,2] <- mdy(as.character(read_page[,2])) # Change values in 'ReleaseDate column into date values 
    read_page[,3] <- as.character(read_page[,3]) # Change values in 'Movie' column into character values
    read_page[,4] <- as.integer(gsub("\\D", "", read_page[,4])) # Change values into integer values
    read_page[,5] <- as.numeric(gsub("\\D", "", read_page[,5])) # Change values into numeric values
    read_page[,6] <- as.numeric(gsub("\\D", "", read_page[,6])) # Change values into numeric values
    colnames(read_page) <- c( # Change columns' names
    "Rank",
    "ReleaseYear",
    "Movie",
    "ProductionBudget",
    "DomesticGross",
    "WorldwideGross")
return(read_page)
}

In [4]:
# Make an empty data frame
movies_df <- data.frame(
    Rank = numeric(),
    ReleaseYear = character(),
    Movie = character(),
    ProductionBudget = integer(),
    DomesticGross = numeric(),
    WorldwideGross = numeric()
)
i = 0 
while (i < 59) { # Combine all data frames which are scraped from each url as one single data frame 
    i = i + 1
    table_temp <- get_movie_df_info(movie_df_url_links[i]) 
    movies_df <- rbind(movies_df, table_temp)
}
movies_df

Warning message:
" 1 failed to parse."Warning message:
" 1 failed to parse."Warning message:
" 1 failed to parse."Warning message:
" 1 failed to parse."Warning message:
" 2 failed to parse."Warning message:
" 1 failed to parse."Warning message:
" 2 failed to parse."Warning message:
" 1 failed to parse."Warning message:
" 2 failed to parse."Warning message:
" 2 failed to parse."Warning message:
" 4 failed to parse."Warning message:
" 4 failed to parse."Warning message:
" 2 failed to parse."Warning message:
" 3 failed to parse."Warning message:
" 1 failed to parse."Warning message:
" 1 failed to parse."Warning message:
" 2 failed to parse."Warning message:
" 1 failed to parse."Warning message:
" 1 failed to parse."Warning message:
" 4 failed to parse."Warning message:
" 2 failed to parse."Warning message:
" 3 failed to parse."Warning message:
" 3 failed to parse."Warning message:
" 2 failed to parse."Warning message:
" 1 failed to parse."Warning message:
" 3 failed to parse."Warning mess

Rank,ReleaseYear,Movie,ProductionBudget,DomesticGross,WorldwideGross
<dbl>,<date>,<chr>,<int>,<dbl>,<dbl>
1,2009-01-07,Avatar,425000000,760507625,2789705275
2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
3,2019-02-03,Avengers: Endgame,400000000,858373000,2795473000
4,2015-02-02,Avengers: Age of Ultron,330600000,459005868,1403013963
5,2017-01-03,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
6,2015-01-06,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220
7,2018-02-05,Avengers: Infinity War,300000000,678815482,2048134200
8,2007-02-04,Pirates of the Caribbean: At World’s End,300000000,309420425,963420425
9,2017-01-03,Justice League,300000000,229024295,655945209


In [5]:
# Write a CSV file with one single data frame which contains all the scraped data from thenumbers.com

write.csv(movies_df, file = "movies_df_1.csv", na="", row.names = F)

In this stage, we separated a movie title by ":" using delimiter function of the Excel for using API to get the 'movieID'. Then we had got 'movies_df_by_year.csv' dataset. However, in this process, the budget column was deleted. It was a hidden problem we had made in the next stage.

#### Let's move to the next work of the Step_1

#### 2. Using API
So far, we had got financial information regarding movies.
#### However,
there is no unique ID which called 'movieID' column for merging with IMDB dataset In the 'movies_df_1.csv' file, So we had decided that using API to get 'movieID'

In [6]:
# load data from csv file scrappped from Numbers.com
movies_numbers_df <- read_csv(file = "movies_df_by_year.csv")

Parsed with column specification:
cols(
  Rank = col_double(),
  ReleaseYear = col_date(format = ""),
  year = col_double(),
  Movie_first = col_character(),
  Movie_second = col_character(),
  DomesticGross = col_double(),
  WorldwideGross = col_double()
)


In [7]:
movies_numbers_df %>% head(10)

Rank,ReleaseYear,year,Movie_first,Movie_second,DomesticGross,WorldwideGross
<dbl>,<date>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
1,2009-12-17,2009,Avatar,NA,760507625,2789705275
2,2011-05-20,2011,Pirates of the Caribbean,On Stranger Tides,241063875,1045663875
3,2019-04-23,2019,Avengers,Endgame,858373000,2795473000
4,2019-06-05,2019,Dark Phoenix,NA,65845974,246356895
5,2015-04-22,2015,Avengers,Age of Ultron,459005868,1403013963
6,2017-12-13,2017,Star Wars Ep. VIII,The Last Jedi,620181382,1316721747
7,2015-12-16,2015,Star Wars Ep. VII,The Force Awakens,936662225,2053311220
8,2018-04-25,2018,Avengers,Infinity War,678815482,2048134200
9,2007-05-24,2007,Pirates of the Caribbean,At World?<U+0080>?s End,309420425,963420425


In [8]:
movies_numbers_df %>%
     skim() %>%
        kable()

Skim summary statistics  
 n obs: 5848    
 n variables: 7    

Variable type: character

|   variable   | missing | complete |  n   | min | max | empty | n_unique |
|--------------|---------|----------|------|-----|-----|-------|----------|
| Movie_first  |    0    |   5848   | 5848 |  1  | 50  |   0   |   5611   |
| Movie_second |  5408   |   440    | 5848 |  2  | 36  |   0   |   415    |

Variable type: Date

|  variable   | missing | complete |  n   |    min     |    max     |   median   | n_unique |
|-------------|---------|----------|------|------------|------------|------------|----------|
| ReleaseYear |   169   |   5679   | 5848 | 1915-02-08 | 2022-12-16 | 2007-02-02 |   2741   |

Variable type: numeric

|    variable    | missing | complete |  n   |  mean   |   sd    |  p0  |    p25    |   p50   |   p75   |  p100   |   hist   |
|----------------|---------|----------|------|---------|---------|------|-----------|---------|---------|---------|----------|
| DomesticGross  |    0

In [9]:
# we had missing values in column year, we decided to exclude them
movies_numbers_df<-movies_numbers_df %>%
 filter(year!=1900)

In [10]:
# to work with API we need library httr
library(httr)

In [11]:
# function which will retrieve info about movie from imdb by title and year
# APIKEY was received after subscription on omdbapi.com
# One API key could get no more than 1000 requests per day!
# Several API keys were activated and used:95fa3c53,e31539fc,9879d6b0,50782ec8,71f53a8c
get_MovieId <- function(p_year, p_title)
{
    #print(p_year)
    #print(p_title)
    # remove white space from band name and substitute them with a +   
    p_title <- str_replace_all(p_title, " ", "+") # this line uses a function from stringr
    #define the url and get info from api
    url<-glue("http://www.omdbapi.com/?t={p_title}&y={p_year}&plot=full&apikey=71f53a8c")
    #print(url)
    r <- GET(url) %>%
      content() 
    #print(r)
    if (r$Response=='True')
   { 
       return(r$imdbID) 
      }
   else
   {
       return("NA")
   }
    
}  

In [12]:
# function to create a new MoviID column , years and titles input vectors from our dataset 
# function returns ids in a column
get_MovieId_Column <- function(years,titles)
{
    ids<-NULL
    for (i in 1:length(years)){
       year<-years[i]
       title<-titles[i]
       id<-get_MovieId(year, title)
       ids<-rbind(ids,id) 
   } 
   return(ids) 
}

In [13]:
# Here we create a new column MovieId,
# Severeal datasets were created because of limitation in 1000 requests.
# Limitation of rows for request was by rank. 
# First, we got first portion of 900 rows and save it in file movies_df_1_900.csv
# then we select rows from 901 to 1800 in file movies_df_900_1800.csv
# from 1801 to 2000 in file movies_df_1800_2000.csv
# from 2001 to 2900 in file movies_df_2001_2900.csv
# from 2901 to 3890 in file movies_df_2901_3890.csv
# from 3891 to 4880 in file movies_df_3891_4880.csv
# from 4881 to 5800 in file movies_df_4881_5800.csv
movies_numbers_df1 <- movies_numbers_df     %>%
    filter(Rank>=4881,Rank<=5870) %>% 
    mutate(movieId=get_MovieId_Column(year, Movie_first))

In [14]:
# save data to file
write_csv(movies_numbers_df1,"movies_df_4881_5800.csv")

In [15]:
# read data from saved file to chek it
movies_dff <- read_csv(file = "movies_df_4881_5800.csv")

Parsed with column specification:
cols(
  Rank = col_double(),
  ReleaseYear = col_date(format = ""),
  year = col_double(),
  Movie_first = col_character(),
  Movie_second = col_character(),
  DomesticGross = col_double(),
  WorldwideGross = col_double(),
  movieId = col_character()
)


#### 3. Merging Datasets
Now we have seven movies_df.csv files which have movieID column.
(For eaxample: movies_df_4881_5800.csv)

#### Eventually,
We have IMDB datasets, Numbers dataset with "MovieID".
So we can do merge those datasets for the next processes.

In [16]:
# Define variables for the separated the Numbers datasets for preparing to merge work.

movies_df_1_900 = read_csv(file = "movies_df_1_900.csv")
movies_df_900_1800 = read_csv(file = "movies_df_900_1800.csv")
movies_df_1800_2000 = read_csv(file = "movies_df_1800_2000.csv")
movies_df_2001_2900 = read_csv(file = "movies_df_2001_2900.csv")
movies_df_2901_3890 = read_csv(file = "movies_df_2901_3890.csv")
movies_df_3891_4880 = read_csv(file = "movies_df_3891_4880.csv")
movies_df_4881_5800 = read_csv(file = "movies_df_4881_5800.csv")

Parsed with column specification:
cols(
  Rank = col_double(),
  ReleaseYear = col_date(format = ""),
  year = col_double(),
  Movie_first = col_character(),
  Movie_second = col_character(),
  DomesticGross = col_double(),
  WorldwideGross = col_double(),
  movieId = col_character()
)
Parsed with column specification:
cols(
  Rank = col_double(),
  ReleaseYear = col_date(format = ""),
  year = col_double(),
  Movie_first = col_character(),
  Movie_second = col_character(),
  DomesticGross = col_double(),
  WorldwideGross = col_double(),
  movieId = col_character()
)
Parsed with column specification:
cols(
  Rank = col_double(),
  ReleaseYear = col_date(format = ""),
  year = col_double(),
  Movie_first = col_character(),
  Movie_second = col_character(),
  DomesticGross = col_double(),
  WorldwideGross = col_double(),
  movieId = col_character()
)
Parsed with column specification:
cols(
  Rank = col_double(),
  ReleaseYear = col_date(format = ""),
  year = col_double(),
  Movie_first 

In [17]:
# Merging the Numbers data which had been separated for getting 'MovieID' into one data frame.

movies_df_1_1800 <- rbind(movies_df_1_900, movies_df_900_1800)
movies_df_1_2000 <- rbind(movies_df_1_1800, movies_df_1800_2000)
movies_df_1_2900 <- rbind(movies_df_1_2000, movies_df_2001_2900)
movies_df_1_3890 <- rbind(movies_df_1_2900, movies_df_2901_3890)
movies_df_1_4880 <- rbind(movies_df_1_3890, movies_df_3891_4880)
the_numbers_movies_finacial_data <- rbind(movies_df_1_4880, movies_df_4881_5800)
the_numbers_movies_finacial_data <- mutate(the_numbers_movies_finacial_data, tconst = movieId)
the_numbers_movies_finacial_data %>%
    head(4)

Rank,ReleaseYear,year,Movie_first,Movie_second,DomesticGross,WorldwideGross,movieId,tconst
<dbl>,<date>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,2009-12-17,2009,Avatar,NA,760507625,2789705275,tt0499549,tt0499549
2,2011-05-20,2011,Pirates of the Caribbean,On Stranger Tides,241063875,1045663875,tt1298650,tt1298650
3,2019-04-23,2019,Avengers,Endgame,858373000,2795473000,tt4154796,tt4154796
4,2019-06-05,2019,Dark Phoenix,NA,65845974,246356895,tt6565702,tt6565702


In [18]:
# Storing the previous data frame into CSV file for the next work.
write_csv(the_numbers_movies_finacial_data,"the_numbers_movies_finacial_data.csv")

In [ ]:
# Define variables for the three datasets of IMDB datasets for preparing to merge work.

title_basics = read_tsv(file = "title_basics.tsv")
title_ratings = read_tsv(file = "title_ratings.tsv")
title_crew = read_tsv(file = "title_crew.tsv")
name_basics = read_tsv(file = "name_basics.tsv")

Parsed with column specification:
cols(
  tconst = col_character(),
  titleType = col_character(),
  primaryTitle = col_character(),
  originalTitle = col_character(),
  isAdult = col_double(),
  startYear = col_double(),
  endYear = col_character(),
  runtimeMinutes = col_character(),
  genres = col_character()
)
Warning message:
"270763 parsing failures.
  row          col           expected actual               file
32547 primaryTitle delimiter or quote        'title_basics.tsv'
32547 primaryTitle delimiter or quote    S   'title_basics.tsv'
32547 primaryTitle delimiter or quote        'title_basics.tsv'
32547 primaryTitle delimiter or quote    G   'title_basics.tsv'
44813 startYear    a double              \N 'title_basics.tsv'
..... ............ .................. ...... ..................
See problems(...) for more details.
"Parsed with column specification:
cols(
  tconst = col_character(),
  averageRating = col_double(),
  numVotes = col_double()
)
Parsed with column specificat

In [ ]:
# Merging 'title_basics.tsv' of the IMDB datasets with the Numbers dataset.
numbers_basic = merge(x = the_numbers_movies_finacial_data, y = title_basics, by = "tconst", all.x = TRUE)

# Merging 'title_ratings.tsv' of the IMDB datasets with the Numbers dataset.
numbers_basic_ratings = merge(x = numbers_basic, y = title_ratings, by = "tconst", all.x = TRUE)

In [37]:
# Mutating a new column which is 'nconst' to merge with 'name_basic.tsv'
title_crew <- mutate(title_crew, nconst = directors)

tconst,directors,writers,nconst
<chr>,<chr>,<chr>,<chr>
tt0000001,nm0005690,\N,nm0005690
tt0000002,nm0721526,\N,nm0721526
tt0000003,nm0721526,\N,nm0721526
tt0000004,nm0721526,\N,nm0721526
tt0000005,nm0005690,\N,nm0005690


In [38]:
# Merging 'name_basic.tsv' of the IMDB datasets with the Numbers dataset.
numbers_basic_ratings_crew = merge(x = numbers_basic_ratings, y = title_crew, by = "tconst", all.x = TRUE)

# Merging 'title_crew.tsv' of the IMDB datasets with the Numbers dataset.
numbers_basic_ratings_crew_names = merge(x = numbers_basic_ratings_crew, y = name_basics, by = "nconst", all.x = TRUE)

tconst,Rank,ReleaseYear,year,Movie_first,Movie_second,DomesticGross,WorldwideGross,movieId,titleType,...,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,nconst
<chr>,<dbl>,<date>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,...,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
tt0004972,5740,1915-02-08,1915,The Birth of a Nation,NA,1e+07,1.1e+07,tt0004972,movie,...,0,1915,\N,195,"Drama,History,War",6.4,20740,nm0000428,"nm0228746,nm0000428,nm0940488",nm0000428
tt0006333,5677,1916-12-24,1916,"20,000 Leagues Under the Sea",NA,8e+06,8.0e+06,tt0006333,movie,...,0,1916,\N,105,"Action,Adventure,Sci-Fi",6.2,1426,nm0665737,"nm0894523,nm0665737",nm0665737
tt0006864,5584,1916-09-05,1916,Intolerance,NA,0e+00,0.0e+00,tt0006864,movie,...,0,1916,\N,163,"Drama,History",7.8,13032,nm0000428,"nm0048512,nm0115218,nm0000428,nm0002616,nm0640437,nm1578667,nm0940488",nm0000428


In [42]:
# Storing the previous data frame into CSV file for the next work in Julia.
write_csv(numbers_basic_ratings_crew_names, "movie_data_step_1_R.csv")

#### Great!
Now we have fully gathered data through the above processes by using R.
In the data wrangling part, we are going to use Julia.

#### find a file name 'project_step_2(data_wrangle)_Julia'